In [1]:
import numpy as np
import xarray as xr
import gcsfs
from tqdm.autonotebook import tqdm
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_2392/270741471.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Regrid monthly mean gridded IBE for cmip6 models to psmsl coordinates:

In [3]:
def get_lonlat_idx_nearest_to_tgs(tg_ds,ds):
    '''tg_ds = xr.DataSet containing 'lon' and 'lat' coordinates of tide gauges
    ds    = xr.DataSet containing CMIP6 data to subset
    '''
    lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    lat_name = list(k for k in ds.dims if 'lat' in k)[0]
    
    #compute distances between TG coordinates and grid cell centers
    distances = 2*np.arcsin( np.sqrt(
        np.sin( (np.pi/180) * 0.5*(ds[lat_name]-tg_ds.lat) )**2 +
        np.cos((np.pi/180)*tg_ds.lat)*np.cos((np.pi/180)*ds[lat_name])*np.sin((np.pi/180)*0.5*(ds[lon_name]-tg_ds.lon))**2) )

    distances = distances.where(np.isfinite(ds.ibe.isel(period=0,month=0,drop=True).mean(dim=('member_id','experiment_id'))),1e30)
    idx_nearest = distances.argmin(dim=[lon_name,lat_name]) #find indices of nearest grid cells
    return idx_nearest
    
input_path = 'gs://leap-persistent/timh37/CMIP6/ibe_1x1_month_means'
output_path = 'gs://leap-persistent/timh37/CMIP6/ibe_psmsl_month_means'

target_grid = xr.open_dataset('../month_means_1993_2022_psmsl.nc')
target_grid.attrs['name'] = 'psmsl'

fns = [k for k in fs.ls(input_path) if 'DS_Store' not in k]

for fn in tqdm(fns):
    ds = xr.open_dataset('gs://'+fn,engine='zarr')
    idx_nearest = get_lonlat_idx_nearest_to_tgs(target_grid,ds)
    
    ds_new = ds.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])

    ##ds_new['tg'] = tgs
    ds_new['lat'] = ('id',target_grid.lat.values)
    ds_new['lon'] = ('id',target_grid.lon.values)
    ds_new.to_zarr(os.path.join(output_path,fn.split('/')[-1]),mode='w')

Regrid gridded IBE for CMIP6 models to psmsl coordinates:

In [ ]:
def get_lonlat_idx_nearest_to_tgs(tg_ds,ds):
    '''tg_ds = xr.DataSet containing 'lon' and 'lat' coordinates of tide gauges
    ds    = xr.DataSet containing CMIP6 data to subset
    '''
    lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    lat_name = list(k for k in ds.dims if 'lat' in k)[0]
    
    #compute distances between TG coordinates and grid cell centers
    distances = 2*np.arcsin( np.sqrt(
        np.sin( (np.pi/180) * 0.5*(ds[lat_name]-tg_ds.lat) )**2 +
        np.cos((np.pi/180)*tg_ds.lat)*np.cos((np.pi/180)*ds[lat_name])*np.sin((np.pi/180)*0.5*(ds[lon_name]-tg_ds.lon))**2) )

    distances = distances.where(np.isfinite(ds.ibe.isel(time=0,drop=True).mean(dim=('member_id'))),1e30)
    idx_nearest = distances.argmin(dim=[lon_name,lat_name]) #find indices of nearest grid cells
    return idx_nearest
    
input_path = 'gs://leap-persistent/timh37/CMIP6/ibe_1x1'
output_path = 'gs://leap-persistent/timh37/CMIP6/ibe_psmsl'

target_grid = xr.open_dataset('../month_means_1993_2022_psmsl.nc')
target_grid.attrs['name'] = 'psmsl'

model_dirs = [k for k in fs.ls(input_path) if 'DS_Store' not in k]

for model_dir in model_dirs:
    fns = [k for k in fs.ls(model_dir) if 'DS_Store' not in k]
    
    for fn in tqdm(fns):
        ds = xr.open_dataset('gs://'+fn,engine='zarr')
        idx_nearest = get_lonlat_idx_nearest_to_tgs(target_grid,ds)
        
        ds_new = ds.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])
    
        ##ds_new['tg'] = tgs
        ds_new['lat'] = ('id',target_grid.lat.values)
        ds_new['lon'] = ('id',target_grid.lon.values)
        ds_new.to_zarr('gs://'+fn.replace('1x1','psmsl'),mode='w',zarr_format=2)

  0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset('gs://'+fn,engine='zarr')
/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset('gs://'+fn,engine='zarr')
/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open

  0%|          | 0/160 [00:00<?, ?it/s]

/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset('gs://'+fn,engine='zarr')
/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open_dataset('gs://'+fn,engine='zarr')
/tmp/ipykernel_2392/1202660612.py:29: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = xr.open

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]